In [34]:
news_data = []
with open("news_summary.txt", "r", encoding="utf-8") as file:
    news_data = file.readlines()
    print(f"loaded {len(news_data)}")

loaded 259


In [35]:
import ollama
embedding_model = "hf.co/CompendiumLabs/bge-base-en-v1.5-gguf"
language_model = "qwen3:1.7b"

vector_db = []

In [36]:
def add_chunk_to_database(chunk):
    embedding = ollama.embed(model=embedding_model, input=chunk)["embeddings"][0]
    vector_db.append((chunk, embedding))

In [28]:
for i, chunk in enumerate(news_data):
    add_chunk_to_database(chunk)
    print(f"added chunk {i+1}/{len(news_data)} to the database")

added chunk 1/178 to the database
added chunk 2/178 to the database
added chunk 3/178 to the database
added chunk 4/178 to the database
added chunk 5/178 to the database
added chunk 6/178 to the database
added chunk 7/178 to the database
added chunk 8/178 to the database
added chunk 9/178 to the database
added chunk 10/178 to the database
added chunk 11/178 to the database
added chunk 12/178 to the database
added chunk 13/178 to the database
added chunk 14/178 to the database
added chunk 15/178 to the database
added chunk 16/178 to the database
added chunk 17/178 to the database
added chunk 18/178 to the database
added chunk 19/178 to the database
added chunk 20/178 to the database
added chunk 21/178 to the database
added chunk 22/178 to the database
added chunk 23/178 to the database
added chunk 24/178 to the database
added chunk 25/178 to the database
added chunk 26/178 to the database
added chunk 27/178 to the database
added chunk 28/178 to the database
added chunk 29/178 to the dat

In [29]:
# use cosine similiarity to find close chunks 
def cosine_similarity(a, b):
  dot_product = sum([x * y for x, y in zip(a, b)])
  norm_a = sum([x ** 2 for x in a]) ** 0.5
  norm_b = sum([x ** 2 for x in b]) ** 0.5
  return dot_product / (norm_a * norm_b)

In [30]:
def retrieve(query, top_n=10):
    query_embedding = ollama.embed(model=embedding_model, input=query)["embeddings"][0]
    similarities = []
    for chunk, embedding in vector_db:
        similarity = cosine_similarity(query_embedding, embedding)
        similarities.append((chunk, similarity))
    # sort by similarity in descending order, because higher similarity means more relevant chunks
    similarities.sort(key=lambda x: x[1], reverse=True)
    # finally, return the top N most relevant chunks
    return similarities[:top_n]

In [31]:
input_query = input("Ask me anything about stocks! ")
retrieved_knowledge = retrieve(input_query)

print("retrieved knowledge: ")
for chunk, similarity in retrieved_knowledge:
    print(f" - (similarity: {similarity:.2f}) {chunk}")

context_text = '\n'.join([f' - {chunk}' for chunk, similarity in retrieved_knowledge])

instruction_prompt = f"""
You are a stock market analysis assistant with access to real-time news via RAG.

You are a stock market analysis assistant with access to real-time news via RAG.

Rules:
1. Use ONLY the retrieved news articles as your information source.
2. Always rely on the LATEST news provided in the feed. Do not speculate or fabricate numbers, prices, or events.
3. Summarize ALL of the retrieved articles together into a single long paragraph. 
   - Do not separate articles.
   - Do not use bullet points, lists, or headings.
   - Weave the information naturally so it reads like one continuous narrative.
4. Expand on the context, potential impacts on stocks, sectors, or the overall market, and sentiment within the same paragraph.
5. Clearly distinguish facts from interpretation by using phrases like “According to the article…” for reporting and “This could suggest…” for analysis.
6. If no useful financial information is present, state that directly instead of fabricating.

Your goal is to return one cohesive, well-written paragraph that combines all recent news into a single flowing analysis, helping users understand how the latest developments may influence stocks or sectors.

Use only the following pieces of context to answer the question. Don't make up any new information:
{context_text}
"""

retrieved knowledge: 
 - (similarity: 0.64) Koss Olinger Consulting LLC increased its stake in Microsoft Corporation (NASDAQ:MSFT – Free Report) by 36.8% in the 1st quarter, Holdings Channel reports. Koss Olinger Consulting LLC’s holdings in Microsoft were worth $5,781,000 at the end of the most recent reporting period. Geode Capital Management LLC now owns 168,128,024 shares of the software giant’s stock valued at $70,720,874,000 after buying an additional 3,103,212 shares during the period. Norges Bank purchased a new stake in Microsoft during the fourth quarter valued at about $44,343,058,000. Following the completion of the sale, the chief executive officer directly owned 790,852 shares in the company, valued at $399,206,272.56.

 - (similarity: 0.63) Banco Santander S.A. now owns 736,654 shares of the software giant’s stock valued at $276,533,000 after buying an additional 63,988 shares during the period. Forge First Asset Management Inc. purchased a new position in Microsoft duri

In [32]:
stream = ollama.chat(
    model=language_model,
    messages=[
        {'role': 'system', 'content': instruction_prompt},
        {'role': 'user', 'content': input_query},
    ],
    stream=True
)

print('Chatbot response:')
for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)

Chatbot response:
<think>
Okay, let me try to figure out how to handle this query about Microsoft. The user just said "microsoft," so I need to check the latest news related to Microsoft. 

First, I'll go through the provided news articles to gather all the relevant information. There are a lot of articles here, so I need to make sure I don't miss any key points. 

Looking at the first article, Koss Olinger Consulting LLC increased their stake in Microsoft by 36.8%, and Norges Bank also bought a new stake. The total value of their holdings is mentioned, which is important for stock analysis. 

Then there's Banco Santander S.A. buying shares, and Forge First Asset Management and Privium Fund Management UK also adding positions. The stock price went down 2.6%, which might affect investor sentiment. 

Summit Financial Strategies Inc. reduced their stake, which could be a factor in the overall market. Wall Street Zen raised the rating to "buy," which is a positive sign. 

Other companies l